In [1]:
import numpy as np
from dataset import house_sales_data

# 8.1 Linear and affine functions

## Matrix-vector product function

Let’s define an instance of the matrix-vector product function, and then numerically check that superposition holds.

In [2]:
A = np.array([[-0.1, 2.8, -1.6],
              [2.3, -0.6, -3.6]])  # 2 by 3 matrix A
f = lambda x: A @ x

# Let's check superposition
x = np.array([1, 2, 3])
y = np.array([-3, -1, 2])
alpha = 0.5
beta = -1.6
LHS = f(alpha*x + beta*y)
print('LHS:', LHS)
RHS = alpha*f(x) + beta*f(y)
print('RHS:', RHS)
print(np.linalg.norm(LHS - RHS))

LHS: [ 9.47 16.75]
RHS: [ 9.47 16.75]
1.7763568394002505e-15


In [3]:
f(np.array([0, 1, 0]))  # Should be second column of A

array([ 2.8, -0.6])

## De-meaning matrix

In [4]:
de_mean = lambda n: np.identity(n) - (1/n)
x = np.array([0.2, 2.3, 1.0])
de_mean(len(x)) @ x #De-mean using matrix multiplication

array([-0.96666667,  1.13333333, -0.16666667])

In [5]:
x - sum(x)/len(x)

array([-0.96666667,  1.13333333, -0.16666667])

In [6]:
x - np.mean(x)

array([-0.96666667,  1.13333333, -0.16666667])

## Examples of functions that are not linear

The componentwise absolute value and the sort function are examples of nonlinear functions.

In [7]:
f = lambda x: abs(x)  # componentwise absolute value
x = np.array([1, 0])
y = np.array([0, 1])
alpha = -1
beta = 2
f(alpha*x + beta*y)

array([1, 2])

In [8]:
alpha*f(x) + beta*f(y)

array([-1,  2])

In [9]:
f = lambda x: np.array(sorted(x, reverse = True))
f(alpha*x + beta*y)

array([ 2, -1])

In [10]:
alpha*f(x) + beta*f(y)

array([1, 0])

## Affine functions

In [11]:
f = lambda x: np.ones((3, x.size)) @ x + np.array([1, 2, 3])  # componentwise absolute value
x = np.array([1, 0])
y = np.array([0, 1])
alpha = -1
beta = 2
f(alpha*x + beta*y)  # alpha + beta = 1

array([2., 3., 4.])

In [12]:
alpha*f(x) + beta*f(y)

array([2., 3., 4.])

In [13]:
alpha = -5
beta = 2
f(alpha*x + beta*y)  # alpha + beta != 1

array([-2., -1.,  0.])

In [14]:
alpha*f(x) + beta*f(y)

array([ -6.,  -9., -12.])

# 8.2 Linear function models

## Price elasticity of demand

Let’s use a price elasticity of demand matrix to predict the demand for three products when the prices are changed a bit. Using this we can predict the change in total profit, given the manufacturing costs.

In [15]:
p = np.array([10, 20, 15])  # Current prices
d = np.array([5.6, 1.5, 8.6])  # Current demand (say in thousands)
c = np.array([6.5, 11.2, 9.8])  # Cost to manufacture
profit = (p - c) @ d  # Current total profit
print(profit)

77.51999999999998


In [16]:
#Demand elesticity matrix
E = np.array([[-0.3, 0.1, -0.1],
              [0.1, -0.5, 0.05],
              [-0.1, 0.05, -0.4]])
p_new = np.array([9, 21, 14])  # Proposed new prices
delta_p = (p_new - p)/p  # Fractional change in prices
print(delta_p)

[-0.1         0.05       -0.06666667]


In [17]:
delta_d = E @ delta_p  # Predicted fractional change in demand
print(delta_d)

[ 0.04166667 -0.03833333  0.03916667]


In [18]:
d_new = d * (1 + delta_d)  # Predicted new demand
print(d_new)

[5.83333333 1.4425     8.93683333]


In [19]:
profit_new = (p_new - c) @ d_new #Predicted new profit
print(profit_new)

66.25453333333333


## Taylor approximation

Consider the nonlinear function $f : R^2 → R^2$ given by
$$
f(x) = \begin{bmatrix}
\left\|x − a\right\| \\
\left\|x − b\right\|
\end{bmatrix}
= \begin{bmatrix}
\sqrt{(x_1 − a_1)^2 + (x_2 − a_2)^2} \\
\sqrt{(x_1 − b_1)^2 + (x_2 − b_2)^2}
\end{bmatrix}
$$

The two components of f gives the distance of x to the points a and b. The function is differentiable, except when x = a or x = b. Its derivative or Jacobian matrix is given by
$$Df(z) = \begin{bmatrix}
\frac{\partial f_1}{\partial x_1}(z) & \frac{\partial f_1}{\partial x_2}(z) \\
\frac{\partial f_2}{\partial x_1}(z) & \frac{\partial f_2}{\partial x_2}(z)
\end{bmatrix}
= \begin{bmatrix}
\frac{z_1 - a_1}{\left\| z - a \right\|} & \frac{z_2 - a_2}{\left\| z - a \right\|} \\
\frac{z_1 - b_1}{\left\| z - b \right\|} & \frac{z_2 - b_2}{\left\| z - b \right\|}
\end{bmatrix}
$$


In [20]:
f = lambda x: np.array([np.linalg.norm(x - a),
                        np.linalg.norm(x - b)])

Df = lambda z: np.array([(z - a) / np.linalg.norm(z - a),
                         (z - b) / np.linalg.norm(z - b)])

f_hat = lambda x: f(z) + Df(z)@(x - z)

a = np.array([1, 0])
b = np.array([1, 1])
z = np.array([0, 0])

f(np.array([0.1, 0.1]))

array([0.90553851, 1.27279221])

In [21]:
f_hat(np.array([0.1, 0.1]))

array([0.9       , 1.27279221])

In [22]:
f(np.array([0.5, 0.5]))

array([0.70710678, 0.70710678])

In [23]:
f_hat(np.array([0.5, 0.5]))

array([0.5       , 0.70710678])

## Regression model

In [24]:
# parameters in regression model
beta = [148.73, -18.85]
v = 54.40
D = house_sales_data()
yd = D['price']  # vector of outcomes
N = len(yd)
X = np.vstack((D['area'], D['beds']))
X.shape

(2, 774)

In [25]:
ydhat = beta @ X + v  # vector of predicted outcomes
rd = yd - ydhat  # vector of predicted errors
np.sqrt(sum(rd**2)/len(rd))  # RMS prediction error

74.84571862623025

In [26]:
# Compare with standard deviation of prices
np.std(yd)

112.78216159756509

# 8.3 Systems of linear equations

## Balancing chemical reactions
We verify the linear balancing equations on page 155 of VMLS, for the simple example of electrolysis of water

In [27]:
R = np.array([2, 1])
P = np.array([[2, 0],
              [0, 2]])

# Check balancing coefficients [2, 2, 1]
coeff = np.array([2, 2, 1])
coeff @ np.vstack((R, -P))

array([0, 0])

Check the equation on page 8.21 in http://vmls-book.stanford.edu/vmls-slides.pdf

$a_1\ Cr_2O^{2−}_{7} + a_2\ Fe^{2+} + a_3\ H^+ → b_1\ Cr^{3+} + b_2\ Fe^{3+} + b_3\ H_2O$

In [28]:
R = np.array([[2, 0, 0],
              [7, 0, 0],
              [0, 1, 0],
              [0, 0, 1],
              [-2, 2, 1]])
P = np.array([[1, 0, 0],
              [0, 0, 1],
              [0, 1, 0],
              [0, 0, 2],
              [3, 3, 0]])

# balancing equations (including a1 = 1 constraint)
A = np.block([[R, -P],
          [1, 0, 0, 0, 0, 0]])
b = np.array([0, 0, 0, 0, 0, 1])
x = np.linalg.solve(A, b)
x

array([ 1.,  6., 14.,  2.,  6.,  7.])

In [29]:
np.hstack((R, -P)) @ x

array([ 6.66133815e-16,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -1.77635684e-15])